# Using the Computational Tools of R to Map the Spread of COVID-19 on a County by County Basis
## David Akinyemi, Margaret Smith, Nicholas Beati
### April 30, 2020

In [1]:
# List packages required for this assignment
packages_to_check <- c("lubridate", "igraph", "plyr", "tidyverse", "RColorBrewer")

# Return list of packages that are not installed
packages <- installed.packages()
packages <- as.data.frame(packages)
packages <- as.character(packages$Package)
need_to_install <- packages_to_check[packages_to_check %in% packages == FALSE]

# Installs required packages
if(length(need_to_install > 0)) {
    for(p in 1:length(need_to_install)) {
        install.packages(need_to_install[p])
    }
}

# Loads required packages for this notebook
lapply(packages_to_check, require, character.only = TRUE)

Loading required package: lubridate

Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date

Loading required package: igraph

Attaching package: ‘igraph’

The following objects are masked from ‘package:lubridate’:

    %--%, union

The following objects are masked from ‘package:stats’:

    decompose, spectrum

The following object is masked from ‘package:base’:

    union

Loading required package: plyr

Attaching package: ‘plyr’

The following object is masked from ‘package:lubridate’:

    here

Loading required package: tidyverse
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.3
✔ tibble  3.0.1     ✔ dplyr   0.8.4
✔ tidyr   1.0.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0
Warning message:
“package ‘tidyr’ was built under R version 3.6.2”Warning message:
“package ‘dplyr’ was built under R version 3.6.2”── Conflicts ────────────────────────────────────────── tidyver

[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

[[4]]
[1] TRUE

[[5]]
[1] TRUE

In [2]:
## STEP 1: Coerces NYT COVID data into one row per county per week

# Read in the COVID data
COVID <- read.csv("us-counties.csv")
#print(head(COVID))

# Keep the date, fips and cases columns
keeps <- c("date", "fips", "cases")
COVID <- COVID[keeps]

# Drop rows for which there is no fips code (fips reads in as NA in this case)
COVID <- COVID[complete.cases(COVID), ]

# Convert dates (which read in as strings) to a date format readable by R
COVID$date <- as.POSIXct(COVID$date)

# Convert dates to "ceiling" - this means all round to the Sunday at the end of the week
COVID$date <- COVID$week <- ceiling_date(COVID$date, unit = "weeks")

# Keep only unique rows of the COVID df
# This means that if the same number is reported for the same place in the same week, only keep one copy
COVID <- unique(COVID)

# Create a list of all weeks (defined by the Sunday at the end of the week)
days.of.week <- weekdays(x=as.Date(seq(7), origin="1950-01-01"))

# Create an empty column that will store the week number
weeks <- data.frame(1:length(unique(COVID$week)), unique(COVID$week))
names(weeks)[1] <- "week_number"
names(weeks)[2] <- "weeks"

# Convert end-of-week-dates in COVID df to a week number (i.e. 2020-01-26 becomes 1, 2020-02-02 becomes 2, etc.)
for(w in 1:length(weeks$weeks)) {
       COVID$week_number[COVID$week == weeks$weeks[w]] <- w
    # Subsets the COVID dataframe to keep only the rows associated with one week from the weeks list, adds week number
}

# Drop the date column
COVID <- subset(COVID, select = -c(date))

# Create a list of all weeks (now converted to numbers)
# Create an empty df to hold the highest count for each county for each week
highest_count <- data.frame()

# Write a for-loop that does the following:
# 1) Subset COVID df into weeks.  
# 2) Subset each week subset df into counties
# 3) Keep only the highest count for each county for each week
# 4) Store the resulting df, which should have just the end count for every week for every county, in a new variable

for(w in 1:length(weeks$weeks)) {
    Working_week <- subset(COVID, COVID$week_number == w)
    Working_week <- (unique(Working_week))
    Working_week <- aggregate(Working_week$cases, by = list(Working_week$fips), max)
    Working_week$week_number <- w
    highest_count <- rbind.fill(highest_count, as.data.frame(Working_week))
}
names(highest_count)[1] <- "fips"
names(highest_count)[2] <- "cases"
highest_count <- as.data.frame(highest_count)
head(highest_count)

,fips,cases,week_number
,<int>,<int>,<int>
1,4013,1,1
2,6037,1,1
3,6059,1,1
4,17031,1,1
5,53061,1,1
6,4013,1,2


In [3]:
## STEP 2: Creates a data frame containing neighboring county

# Read in neighboring county data
NEIGHBORS <- read.csv("neighborcounties.csv")

# Drop all neighbor pairs where one of each pair does not appear in all_counties
NEIGHBORS <- NEIGHBORS[NEIGHBORS$orgfips %in% COVID$fips,]
NEIGHBORS <- NEIGHBORS[NEIGHBORS$adjfips %in% COVID$fips,]

In [4]:
## STEP 3: Creates a dataframe containing cumulative cases for each county each week

# Store the total number of weeks in a variable
totalweeks <- max(weeks$week_number)

# Store all county codes as a list
all_county_codes <- unique(highest_count$fips)

# Arrange counties in ascending order by fips code
all_county_codes <- all_county_codes[order(all_county_codes)]

# Create an empty dataframe with the same number of rows as counties, same number of columns as weeks
CC_matrix <- as.data.frame(matrix(nrow=length(all_county_codes), ncol=totalweeks, data <- 0))
colnames(CC_matrix) <- c(1:totalweeks)

# Attach counties as row names
row.names(CC_matrix) <- all_county_codes

# Loop through all of the weeks to count total cases per county each week
for(w in 1:totalweeks) {
    subset_week <- highest_count[(highest_count$week_number == w),]     # Subset the df produced in step 1 to just one week
    for(r in 1:nrow(subset_week)) {     # Loop through the rows in the subset
        county_id <- as.numeric(subset_week$fips[r])         # Store the county code in a variable
        number_cases <- as.numeric(subset_week$cases[r])     # Store the number of cases in a variable
        CC_matrix[which(as.numeric(row.names(CC_matrix)) == county_id), w] <- number_cases # Store the number of cases at the intersection of the county code and the week
    }
}

# Create a new df that contains only new cases each week 
CC_matrix_diff <-matrix(c(NA), ncol=ncol(CC_matrix), nrow=nrow(CC_matrix), byrow=TRUE)
cols <- ncol(CC_matrix) #14 should be the number of weeks
cols_minus_1 <- ncol(CC_matrix) - 1
rows <- nrow(CC_matrix) #Should be 2thousandish, the amount of counties
rownames(CC_matrix_diff)<-1:rows;colnames(CC_matrix_diff)<-1:cols

# Loop through each column of your cumulative totals df to keep only new cases of COVID
# (the number of new cases in week 2 will be the total number measured in week 2, minus the number measured in week 1)
for(i in 1:rows){for(j in 1:cols_minus_1){CC_matrix_diff[i,j+1]<-(as.numeric((CC_matrix)[i,j+1])-as.numeric((CC_matrix)[i,j]))}}
CC_matrix_diff <- as.data.frame(CC_matrix_diff)
CC_matrix_diff$"1" <- CC_matrix$"1"
CC_matrix_diff <- as.matrix(CC_matrix_diff)
rownames(CC_matrix_diff)<- rownames(CC_matrix)

In [5]:
## STEP 4: Produces an edgelist showing whether transmission happened in a two-week period

# Create a df containing two adjacent weeks of data
first_2_weeks <- CC_matrix_diff[,1:2]

# Create a df that holds all of the pairs of neighboring counties 
NEIGHBORS <- NEIGHBORS[c("orgfips", "adjfips")]

COVID_pairs <- as.data.frame(NEIGHBORS)
# Add a new column to the new dataframe to track whether there was transmission
COVID_pairs$transmission <- ""

In [6]:
# Loop through all neighbors to see which pairs meet the following conditions:
# 1) the original county (orgfips) had an increase in the number of cases of COVID in week 1
# 2) the adjacent county (adjfips) had an increase in the number of cases of COVID in week 2
for(n in 1:nrow(COVID_pairs)) {
    # Store the fips of the original county
    ## Wrap this in as.numeric, and use the column name orgfips
    orgfips <- as.numeric(COVID_pairs$orgfips[n])
    
    # Store the number of new cases for the original county in week 1
    new_cases_w1 <- as.numeric(first_2_weeks[which(row.names(first_2_weeks) == orgfips),1])
   
    # Store the fips of the adjacent county
    # Wrap this in as.numeric, and use the column name adjfips
    adjfips <- COVID_pairs$adjfips[n]
    
    # Store the number of new cases for the original county in week 2
    new_cases_w2 <- as.numeric(first_2_weeks[which(row.names(first_2_weeks) == adjfips),2])
    
    # Check whether there were new cases last week in the original county, and new cases this week in the adjacent county
    transmission <- new_cases_w1 > 0 & new_cases_w2 > 0
    
    # Store the resulting boolean in the transmission column
    COVID_pairs$transmission[n] <- transmission
}
# Subset dataframe, keeping only the edges connected by whether there was transmission
COVID_pairs <- COVID_pairs[which(COVID_pairs$transmission == TRUE),]
COVID_pairs <- COVID_pairs[,1:2]
COVID_pairs <- as.matrix(COVID_pairs)

In [7]:
##Now, if you want a function that does all of that, here is the structure

COVID_week <- function(first_week, second_week) {
    
    week_pair <- CC_matrix_diff[,first_week:second_week]
    
    COVID_pairs <- as.data.frame(NEIGHBORS)
    COVID_pairs$transmission <- ""
    
    for(n in 1:nrow(COVID_pairs)) {
        orgfips <- as.numeric(COVID_pairs$orgfips[n])
        new_cases_w1 <- as.numeric(week_pair[which(row.names(week_pair) == orgfips),1])
        adjfips <- COVID_pairs$adjfips[n]
        new_cases_w2 <- as.numeric(week_pair[which(row.names(week_pair) == adjfips),2])
        transmission <- new_cases_w1 > 0 & new_cases_w2 > 0
        COVID_pairs$transmission[n] <- transmission
    }

    COVID_pairs <- COVID_pairs[which(COVID_pairs$transmission == TRUE),]
    COVID_pairs <- COVID_pairs[,1:2]
    COVID_pairs <- as.matrix(COVID_pairs)
    row.names(COVID_pairs) <- NULL
    return(COVID_pairs)
}

In [8]:
#Choosing 3 2-week periods that we thought would be best to investigate to view the change
nrow(COVID_week(5,6))
nrow(COVID_week(10,11))
nrow(COVID_week(13,14))

[1] 1

[1] 8907

[1] 5667

In [9]:
plot_the_week_period <- function(COVID_week) {
    labels <- thenames$county_state[(as.data.frame(COVID_week))$orgfips %in% thenames$fips]
    
    m <- mapply(COVID_week, FUN=as.character)
    COVID_week <- matrix(data=m, ncol=ncol(COVID_week), nrow=nrow(COVID_week))
    
    COVID_week.g <- graph_from_edgelist(COVID_week, directed = TRUE)
    e.wt <- edge_attr(COVID_week.g, "weight")
    
    layout_for_g = layout_with_lgl(COVID_week.g)
    
    filename <- paste(COVID_week, ".png", sep = "")
    
    png(file= filename, width=4000, height=4000)
    plot(COVID_week.g, layout = layout_for_g, vertex.size=1, edge.width = e.wt * 0.001,vertex.label=labels)
    dev.off()
    return 
}

In [10]:
# Produces dataframes for each two-week block
week_5_6 <- COVID_week(5,6)
week_6_7 <- COVID_week(6,7)
week_7_8 <- COVID_week(7,8)
week_8_9 <- COVID_week(8,9)
week_9_10 <- COVID_week(9,10)
week_10_11 <- COVID_week(10,11)
week_11_12 <- COVID_week(11,12)
week_12_13 <- COVID_week(12,13)
week_13_14 <- COVID_week(13,14)

In [11]:
# Produces edge lists for each two-week block
# Starting with week 5 to 6 because there were not new cases in proximate counties before that
week_5_6.g <- graph_from_edgelist(week_5_6, directed = TRUE)
week_6_7.g <- graph_from_edgelist(week_6_7, directed = TRUE)
week_7_8.g <- graph_from_edgelist(week_7_8, directed = TRUE)
week_8_9.g <- graph_from_edgelist(week_8_9, directed = TRUE)
week_9_10.g <- graph_from_edgelist(week_9_10, directed = TRUE)
week_10_11.g <- graph_from_edgelist(week_10_11, directed = TRUE)
week_11_12.g <- graph_from_edgelist(week_11_12, directed = TRUE)
week_12_13.g <- graph_from_edgelist(week_12_13, directed = TRUE)
week_13_14.g <- graph_from_edgelist(week_13_14, directed = TRUE)

In [12]:
# Creates a dataframe containing one row for each county fips, and the state that county is in
county_states <- read.csv("us-counties.csv", header = TRUE)
county_states <- county_states[,c("state", "fips")]
county_states <- unique(county_states)
county_states <- county_states[order(county_states$fips),]
county_states$fips <- as.character(county_states$fips)

In [13]:
# Creates a vector of individual colors of length 50 (for 50 U.S. states)
# Each time you run this cell, you will get a different sample of 50 colors

qual_col_pals <- brewer.pal.info[brewer.pal.info$category == 'qual',] # Lists all Color Brewer palettes that are qualitative
col_vector <- unlist(mapply(brewer.pal, qual_col_pals$maxcolors, rownames(qual_col_pals))) # Extracts all of the colors from all of the qualitative palettes
col_vector <- sample(col_vector, 50) # Samples 50 of those colors

In [14]:
m <- mapply(week_5_6, FUN=as.character)
week_5_6 <- matrix(data=m, ncol=ncol(week_5_6), nrow=nrow(week_5_6))
week_5_6.g <- graph_from_edgelist(week_5_6, directed = TRUE)

In [15]:
m <- mapply(week_6_7, FUN=as.character)
week_6_7 <- matrix(data=m, ncol=ncol(week_6_7), nrow=nrow(week_6_7))
week_6_7.g <- graph_from_edgelist(week_6_7, directed = TRUE)

In [16]:
m <- mapply(week_7_8, FUN=as.character)
week_7_8 <- matrix(data=m, ncol=ncol(week_7_8), nrow=nrow(week_7_8))
week_7_8.g <- graph_from_edgelist(week_7_8, directed = TRUE)

In [17]:
m <- mapply(week_8_9, FUN=as.character)
week_8_9 <- matrix(data=m, ncol=ncol(week_8_9), nrow=nrow(week_8_9))
week_8_9.g <- graph_from_edgelist(week_8_9, directed = TRUE)

In [18]:
m <- mapply(week_9_10, FUN=as.character)
week_9_10 <- matrix(data=m, ncol=ncol(week_9_10), nrow=nrow(week_9_10))
week_9_10.g <- graph_from_edgelist(week_9_10, directed = TRUE)

In [19]:
m <- mapply(week_10_11, FUN=as.character)
week_10_11 <- matrix(data=m, ncol=ncol(week_10_11), nrow=nrow(week_10_11))
week_10_11.g <- graph_from_edgelist(week_10_11, directed = TRUE)

In [20]:
m <- mapply(week_11_12, FUN=as.character)
week_11_12 <- matrix(data=m, ncol=ncol(week_11_12), nrow=nrow(week_11_12))
week_11_12.g <- graph_from_edgelist(week_11_12, directed = TRUE)

In [21]:
m <- mapply(week_12_13, FUN=as.character)
week_12_13 <- matrix(data=m, ncol=ncol(week_12_13), nrow=nrow(week_12_13))
week_12_13.g <- graph_from_edgelist(week_12_13, directed = TRUE)

In [22]:
m <- mapply(week_13_14, FUN=as.character)
week_13_14 <- matrix(data=m, ncol=ncol(week_13_14), nrow=nrow(week_13_14))
week_13_14.g <- graph_from_edgelist(week_13_14, directed = TRUE)

In [23]:
# The match() function compares two vectors, and locates the position of each item in the first vector in the second vector.
# So, running this cell would tell you what rows of county_states match the fips codes in the week_5_6.g network
# (Remember that the V() function returns the vertices (nodes) in a network)

# However, we want the names of the states, not just their positions in the list.
# To get the state names, we subset the state column of the county_states df using the indices returned above:

# To get the state names, we subset the state column of the county_states df using the indices returned above:
V(week_5_6.g)$state <- as.character(county_states$state[match(V(week_5_6.g)$name,county_states$fips)])
V(week_6_7.g)$state <- as.character(county_states$state[match(V(week_6_7.g)$name,county_states$fips)])
V(week_7_8.g)$state <- as.character(county_states$state[match(V(week_7_8.g)$name,county_states$fips)])
V(week_8_9.g)$state <- as.character(county_states$state[match(V(week_8_9.g)$name,county_states$fips)])
V(week_9_10.g)$state <- as.character(county_states$state[match(V(week_9_10.g)$name,county_states$fips)])
V(week_10_11.g)$state <- as.character(county_states$state[match(V(week_10_11.g)$name,county_states$fips)])
V(week_11_12.g)$state <- as.character(county_states$state[match(V(week_11_12.g)$name,county_states$fips)])
V(week_12_13.g)$state <- as.character(county_states$state[match(V(week_12_13.g)$name,county_states$fips)])
V(week_13_14.g)$state <- as.character(county_states$state[match(V(week_13_14.g)$name,county_states$fips)])

In [24]:
# Now we want to create a dataframe that holds the name of each state, and a color associated with that state
state_colors <- as.data.frame(matrix(ncol = 2, nrow = 50)) # Creates empty df
colnames(state_colors) <- c("state", "color") # names columns
state_colors$state <- state.name # uses the variable state.name (which is the names of all U.S. states) to populate one column
state_colors$color <- col_vector # uses the colors we samples above to populate the second column

In [25]:
# Now, we can match the states in each time-slice graph with a color, using the same structure we used above:
V(week_5_6.g)$color <- as.character(state_colors$color[match(V(week_5_6.g)$state,state_colors$state)])
V(week_6_7.g)$color <- as.character(state_colors$color[match(V(week_6_7.g)$state,state_colors$state)])
V(week_7_8.g)$color <- as.character(state_colors$color[match(V(week_7_8.g)$state,state_colors$state)])
V(week_8_9.g)$color <- as.character(state_colors$color[match(V(week_8_9.g)$state,state_colors$state)])
V(week_9_10.g)$color <- as.character(state_colors$color[match(V(week_9_10.g)$state,state_colors$state)])
V(week_10_11.g)$color <- as.character(state_colors$color[match(V(week_10_11.g)$state,state_colors$state)])
V(week_11_12.g)$color <- as.character(state_colors$color[match(V(week_11_12.g)$state,state_colors$state)])
V(week_12_13.g)$color <- as.character(state_colors$color[match(V(week_12_13.g)$state,state_colors$state)])
V(week_13_14.g)$color <- as.character(state_colors$color[match(V(week_13_14.g)$state,state_colors$state)])

In [68]:
# Now we can plot the connections and add a legend
png("week_5-6.png", width = 2000, height = 2000)
plot(week_5_6.g, cex.main = 20, vertex.label = NA, vertex.size = 2, vertex.frame.color = NA, edge.arrow.size = 0.1, layout = layout_with_kk)
legend("bottomleft", state_colors$state, pch=21, col="#777777", pt.bg=state_colors$color, pt.cex=2, cex=2, bty="n", ncol=1)
dev.off()

png 
  2

In [53]:
# Now we can plot the connections and add a legend
png("week_6_7.png", width = 4000, height = 4000)
plot(week_6_7.g, main = "Week 6-7", cex.main =5, vertex.label = NA, vertex.size = 1, vertex.frame.color = NA, edge.arrow.size = 0.1, layout = layout_with_kk)
legend("topleft", state_colors$state, pch=21, col="#777777", pt.bg=state_colors$color, pt.cex=2, cex=.8, bty="n", ncol=1)
dev.off()

png 
  2

In [28]:
# Now we can plot the connections and add a legend
png("week_7_8.png", width = 4000, height = 4000)
plot(week_7_8.g, main = "Week 7-8", cex.main =5, vertex.label = NA, vertex.size = 0.5, vertex.frame.color = NA, edge.arrow.size = 0.1, layout = layout_with_kk)
legend("topleft", state_colors$state, pch=21, col="#777777", pt.bg=state_colors$color, pt.cex=2, cex=.8, bty="n", ncol=1)
dev.off()

png 
  2

In [29]:
# Now we can plot the connections and add a legend
png("week_8_9.png", width = 4000, height = 4000)
plot(week_8_9.g, main = "Week 8-9", cex.main =5, vertex.label = NA, vertex.size = 0.5, vertex.frame.color = NA, edge.arrow.size = 0.1, layout = layout_with_kk)
legend("topleft", state_colors$state, pch=21, col="#777777", pt.bg=state_colors$color, pt.cex=2, cex=.8, bty="n", ncol=1)
dev.off()

png 
  2

In [30]:
# Now we can plot the connections and add a legend
png("week_9_10.png", width = 4000, height = 4000)
plot(week_9_10.g, main = "Week 9-10", cex.main =5, vertex.label = NA, vertex.size = 0.5, vertex.frame.color = NA, edge.arrow.size = 0.1, layout = layout_with_kk)
legend("topleft", state_colors$state, pch=21, col="#777777", pt.bg=state_colors$color, pt.cex=2, cex=.8, bty="n", ncol=1)
dev.off()

png 
  2

In [73]:
# Now we can plot the connections and add a legend
png("week_10_11.png", width = 2000, height = 2000)
plot(week_10_11.g, cex.main =5, vertex.label = NA, vertex.size = 2, vertex.frame.color = NA, edge.arrow.size = 0.1, layout = layout_with_kk)
legend("topleft", state_colors$state, pch=21, col="#777777", pt.bg=state_colors$color, pt.cex=2, cex=2, bty="n", ncol=1)
dev.off()

png 
  2

In [32]:
# Now we can plot the connections and add a legend
png("week_11_12.png", width = 4000, height = 4000)
plot(week_11_12.g, main = "Week 11-12", cex.main =5, vertex.label = NA, vertex.size = 0.5, vertex.frame.color = NA, edge.arrow.size = 0.1, layout = layout_with_kk)
legend("topleft", state_colors$state, pch=21, col="#777777", pt.bg=state_colors$color, pt.cex=2, cex=.8, bty="n", ncol=1)
dev.off()

png 
  2

In [33]:
# Now we can plot the connections and add a legend
png("week_12_13.png", width = 4000, height = 4000)
plot(week_12_13.g, main = "Week 12-13", cex.main =5, vertex.label = NA, vertex.size = 0.5, vertex.frame.color = NA, edge.arrow.size = 0.1, layout = layout_with_kk)
legend("topleft", state_colors$state, pch=21, col="#777777", pt.bg=state_colors$color, pt.cex=2, cex=.8, bty="n", ncol=1)
dev.off()

png 
  2

In [74]:
# Now we can plot the connections and add a legend
png("week_13_14.png", width = 2000, height = 2000)
plot(week_13_14.g, cex.main =5, vertex.label = NA, vertex.size = 2, vertex.frame.color = NA, edge.arrow.size = 0.1, layout = layout_with_kk)
legend("topleft", state_colors$state, pch=21, col="#777777", pt.bg=state_colors$color, pt.cex=2, cex=2, bty="n", ncol=1)
dev.off()

png 
  2

In [35]:
png("all_weeks.png", width = 4000, height = 4000)
par(mfrow=c(3,3), oma = c(0,0,10,0),  mar=c(10,20,10,10), cex.main =5)
plot(week_5_6.g, main = "Week 5-6", cex.main =5, vertex.label = NA, vertex.size = 1, vertex.frame.color = NA, edge.arrow.size = 0.1, layout = layout_with_kk)
box(col="black")
legend("topleft", inset = c(-0.15, 0), state_colors$state, pch=21, col="#777777", pt.bg=state_colors$color, pt.cex=2, cex=2, bty="n", ncol=1)
plot(week_6_7.g, main = "Week 6-7", cex.main = 5, vertex.label = NA, vertex.size = 1, vertex.frame.color = NA, edge.arrow.size = 0.1, layout = layout_with_kk)
box(col="black")
legend("topleft", inset = c(-0.15, 0), state_colors$state, pch=21, col="#777777", pt.bg=state_colors$color, pt.cex=2, cex=2, bty="n", ncol=1)
plot(week_7_8.g, main = "Week 7-8", cex.main = 5, vertex.label = NA, vertex.size = 1, vertex.frame.color = NA, edge.arrow.size = 0.1, layout = layout_with_kk)
box(col="black")
legend("topleft", inset = c(-0.15, 0), state_colors$state, pch=21, col="#777777", pt.bg=state_colors$color, pt.cex=2, cex=2, bty="n", ncol=1)
plot(week_8_9.g, main = "Week 8-9", cex.main = 5, vertex.label = NA, vertex.size = 1, vertex.frame.color = NA, edge.arrow.size = 0.1, layout = layout_with_kk)
box(col="black")
legend("topleft", inset = c(-0.15, 0), state_colors$state, pch=21, col="#777777", pt.bg=state_colors$color, pt.cex=2, cex=2, bty="n", ncol=1)
plot(week_9_10.g, main = "Week 9-10", cex.main = 5, vertex.label = NA, vertex.size = 1, vertex.frame.color = NA, edge.arrow.size = 0.1, layout = layout_with_kk)
box(col="black")
legend("topleft", inset = c(-0.15, 0), state_colors$state, pch=21, col="#777777", pt.bg=state_colors$color, pt.cex=2, cex=2, bty="n", ncol=1)
plot(week_10_11.g, main = "Week 10-11", cex.main = 5, vertex.label = NA, vertex.size = 1, vertex.frame.color = NA, edge.arrow.size = 0.1, layout = layout_with_kk)
box(col="black")
legend("topleft", inset = c(-0.15, 0), state_colors$state, pch=21, col="#777777", pt.bg=state_colors$color, pt.cex=2, cex=2, bty="n", ncol=1)
plot(week_11_12.g, main = "Week 11-12", cex.main = 5, vertex.label = NA, vertex.size = 1, vertex.frame.color = NA, edge.arrow.size = 0.1, layout = layout_with_kk)
box(col="black")
legend("topleft", inset = c(-0.15, 0), state_colors$state, pch=21, col="#777777", pt.bg=state_colors$color, pt.cex=2, cex=2, bty="n", ncol=1)
plot(week_12_13.g, main = "Week 12-13", cex.main = 5, vertex.label = NA, vertex.size = 1, vertex.frame.color = NA, edge.arrow.size = 0.1, layout = layout_with_kk)
box(col="black")
legend("topleft", inset = c(-0.15, 0), state_colors$state, pch=21, col="#777777", pt.bg=state_colors$color, pt.cex=2, cex=2, bty="n", ncol=1)
plot(week_13_14.g, main = "Week 13-14", cex.main = 5, vertex.label = NA, vertex.size = 1, vertex.frame.color = NA, edge.arrow.size = 0.1, layout = layout_with_kk)
box(col="black")
legend("topleft", inset = c(-0.15, 0), state_colors$state, pch=21, col="#777777", pt.bg=state_colors$color, pt.cex=2, cex=2, bty="n", ncol=1)
mtext("The Network Spread of COVID-19", outer = TRUE, cex = 6)
dev.off()

png 
  2

# Week 5-6 Summary

In [36]:
#the g size is the number edges in the network
gsize(week_5_6.g)

#vcount tells us the number of nodes, vertexes, or actors in the network
vcount(week_5_6.g)

#transitivity number of closed triples compared to the number of potential closed triples
transitivity(week_5_6.g)

#the diameter is the overall size of a network, the distance between the two farthest nodes
diameter(week_5_6.g)

#density is the number of ties in the network compared to the number of possible ties
edge_density(week_5_6.g)

#components are subgroups or communities. all actors are connected directly or indirectly
components(week_5_6.g)

[1] 1

[1] 2

[1] NaN

[1] 1

[1] 0.5

$membership
6067 6095 
   1    1 

$csize
[1] 2

$no
[1] 1

In [37]:
##Betweenness

#This line calculates the betweenness of each county for our week 10 period
btwn.wk10 <- betweenness(week_5_6.g)

#This line assigns the corresponding name with its corresponding betweenness value
names(btwn.wk10) <- V(week_5_6.g)#county_states2$county[county_states2$fips %in% rownames(as.matrix(V(week_6_7.g)))]

#This line now rearrages the data in decreasing order of betweennees value
ind <- order(-btwn.wk10)

#This line allow us to see the top ten highest counties with betweenness values 
btwn.wk10[ind][1:10]

#The betweenness value is a value that allows us to see how often a county can be used as a link or gateway 
#between another county within the network

1    2 <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> 
   0    0   NA   NA   NA   NA   NA   NA   NA   NA

In [38]:
## Centrality 

#This line calculates the centrality of each faculty member within our initial data set
cent.eig <- evcent(week_5_6.g)

#This line assigns the corresponding name with its corresponding centrality value
names(cent.eig$vector) <- V(week_5_6.g) #county_states2$county[county_states2$fips %in% rownames(as.matrix(V(week_6_7.g)))]

#This line now rearrages the data in decreasing order of centrality value
ind <- order(-cent.eig$vector)

#This line allow us to see the top ten highest faculty members with centrality values 
cent.eig$vector[ind][1:10]

1    2 <NA> <NA> <NA> <NA> <NA> <NA> <NA> <NA> 
   1    1   NA   NA   NA   NA   NA   NA   NA   NA

# Week 10-11 Summary

In [39]:
#the g size is the number edges in the network
gsize(week_10_11.g)

#vcount tells us the number of nodes, vertexes, or actors in the network
vcount(week_10_11.g)

#transitivity number of closed triples compared to the number of potential closed triples
transitivity(week_10_11.g)

#the diameter is the overall size of a network, the distance between the two farthest nodes
diameter(week_10_11.g)

#density is the number of ties in the network compared to the number of possible ties
edge_density(week_10_11.g)

#components are subgroups or communities. all actors are connected directly or indirectly
components(week_10_11.g)

[1] 8907

[1] 2363

[1] 0.3684345

[1] 87

[1] 0.001595834

$membership
 1001  1021  1047  1051  1085  1101  1003  1025  1053  1097  1099  1129 12033 
    1     1     1     1     1     1     1     1     1     1     1     1     1 
 1009  1043  1055  1073  1095  1115  1127  1011  1005  1087  1109  1113  1013 
    1     1     1     1     1     1     1     1     1     1     1     1     1 
 1035  1039  1041  1131  1015  1019  1029  1121  1017  1081  1111  1123 13145 
    1     1     1     1     1     1     1     1     1     1     1     1     1 
13285  1049 13055 13115 13233  1007  1037  1105  1117  1023  1091  1119 28023 
    1     1     1     1     1     1     1     1     1     1     1     1     1 
28075 28153  1027 13045 13143  1033  1059  1077  1079 28141  1031 12091 12131 
    1     1     1     1     1     1     1     1     1     1     1     1     1 
 1103  1133  1071 13083 13295 12113  1093 28057  1063  1065  1107  1125  1069 
    1     1     1     1     1     1     1     1     1     1     1     1     1 
 1045  1067 12063 13099 13253  1089 47051 47115  1083 47055 47071 47099 47181 
    1     1     1     1     1     1     1     1     1     1     1     1     1 
13215 47103  1057  1075 28095 28039 28059 28087 28103 13149 13053 13259  2020 
    1     1     1     1     1     1     1     1     1     1     1     1     2 
 2122  2170  2090  2240  2290  4001  4011  4017 35003 35006 35031 35045 49037 
    2     2     2     2     2     3     3     3     3     3     3     3     3 
 4003  4019  4023  4005  4007  4015  4025 49025  4013  4021  4009  4012  4027 
    3     3     3     3     3     3     3     3     3     3     3     3     3 
 6025  6065  6071 32003 49053  5001  5041  5069  5079  5085  5095  5107  5117 
    3     3     3     3     3     1     1     1     1     1     1     1     1 
 5003  5017  5043  5139 22067  5005  5129  5137  5007  5015  5143 29119 40001 
    1     1     1     1     1     1     1     1     1     1     1     1     1 
40041  5011  5025 28011 28151  5019  5039  5059  5103  5023  5045  5063  5141 
    1     1     1     1     1     1     1     1     1     1     1     1     1 
 5145  5053  5027  5073 22027 22119  5029  5105  5115  5149  5031  5055  5093 
    1     1     1     1     1     1     1     1     1     1     1     1     1 
 5111 29069  5033  5131 40135  5035  5037  5077  5123 28033 28143 47157 47167 
    1     1     1     1     1     1     1     1     1     1     1     1     1 
 5119  5051  5125  5121  5057  5061  5091  5113  5133  5135  5071  5101  5075 
    1     1     1     1     1     1     1     1     1     1     1     1     1 
 5099  5109  5127 40089  5009 22111  5147  6001  6013  6075  6077  6081  6085 
    1     1     1     1     1     1     1     3     3     3     3     3     3 
 6099  6007  6011  6021  6063  6101  6103  6115  6009  6003  6005  6113  6041 
    3     3     3     3     3     3     3     3     3     3     3     3     3 
 6067  6095  6017  6061 32005  6019  6027  6031  6039  6047  6051  6053  6069 
    3     3     3     3     3     3     3     3     3     3     3     3     3 
 6107  6045  6023  6015  6073  6029 32023  6037  6079  6083  6111  6059  6097 
    3     3     3     3     3     3     3     3     3     3     3     3     3 
32019  6087  6055  6057 32031 32510  6089  6093 41029 41033 41035  6109  8001 
    3     3     3     3     3     3     3     3     3     3     3     3     3 
 8005  8014  8031  8059  8087  8123  8003  8023  8105  8109  8035  8039  8009 
    3     3     3     3     3     3     3     3     3     3     3     3     3 
 8071  8013  8049  8069  8015  8043  8051  8065  8093  8097 35055  8029  8077 
    3     3     3     3     3     3     3     3     3     3     3     3     3 
 8085  8041  8119  8037  8045  8107  8117  8101  8027  8103 49019  8091  8055 
    3     3     3     3     3     3     3     3     3     3     3     3     3 
 8019  8067  8007  8083 56001 56021  8073  8063  8075  8095 31105  8079  8081 
    3     3     3     3     3     3     3     3     3     3     3     3     3 
56007 56037  8113  8089  8121  8125  9

In [40]:
##Betweenness

#This line calculates the betweenness of each county for our week 10 period
btwn.wk10 <- betweenness(week_10_11.g)

#This line assigns the corresponding name with its corresponding betweenness value
names(btwn.wk10) <- V(week_10_11.g)#county_states2$county[county_states2$fips %in% rownames(as.matrix(V(week_6_7.g)))]

#This line now rearrages the data in decreasing order of betweennees value
ind <- order(-btwn.wk10)

#This line allow us to see the top ten highest counties with betweenness values 
btwn.wk10[ind][1:10]

#The betweenness value is a value that allows us to see how often a county can be used as a link or gateway 
#between another county within the network

777     1010      994      808      794      755      795     1974 
636682.5 636017.0 635776.6 634809.9 634371.5 633922.9 633166.4 618389.8 
     757      995 
497481.9 495374.6

In [49]:
V(week_10_11.g)[777]

+ 1/2363 vertex, named, from 26b213f:
[1] 18173

In [50]:
## Centrality 

#This line calculates the centrality of each faculty member within our initial data set
cent.eig <- evcent(week_10_11.g)

#This line assigns the corresponding name with its corresponding centrality value
names(cent.eig$vector) <- V(week_10_11.g) #county_states2$county[county_states2$fips %in% rownames(as.matrix(V(week_6_7.g)))]

#This line now rearrages the data in decreasing order of centrality value
ind <- order(-cent.eig$vector)

#This line allow us to see the top ten highest faculty members with centrality values 
cent.eig$vector[ind][1:10]

1880      1497      1140      1507      1901      1135      1876      1878 
1.0000000 0.9323706 0.9310205 0.9177318 0.8750534 0.8652985 0.8610202 0.8575747 
      355      1128 
0.8569181 0.8222363

In [51]:
V(week_10_11.g)[1880]

+ 1/2363 vertex, named, from 26b213f:
[1] 42107

# Week 13-14 Summary

In [42]:
#the g size is the number edges in the network
gsize(week_13_14.g)

#vcount tells us the number of nodes, vertexes, or actors in the network
vcount(week_13_14.g)

#transitivity number of closed triples compared to the number of potential closed triples
transitivity(week_13_14.g)

#the diameter is the overall size of a network, the distance between the two farthest nodes
diameter(week_13_14.g)

#density is the number of ties in the network compared to the number of possible ties
edge_density(week_13_14.g)

#components are subgroups or communities. all actors are connected directly or indirectly
components(week_13_14.g)

[1] 5667

[1] 1991

[1] 0.3299233

[1] 42

[1] 0.001430306

$membership
 1001  1021  1047  1051  1085  1101  1003  1053  1097 12033  1005  1011  1045 
    1     1     1     1     1     1     1     1     1     1     1     1     1 
 1067  1109  1113 13259  1007  1065  1073  1117  1009  1055  1095  1115  1127 
    1     1     1     1     1     1     1     1     1     1     1     1     1 
 1087  1013  1039  1131  1015  1121  1017  1081  1111 13145 13285  1019  1049 
    1     1     1     1     1     1     1     1     1     1     1     1     1 
13115 13233  1037  1023  1091  1119 28023 28075 28153  1025  1027  1031  1033 
    1     1     1     1     1     1     1     1     1     1     1     1     1 
 1077  1079  1035  1041  1043  1103  1069  1071 13083 13295 12113  1059  1093 
    1     1     1     1     1     1     1     1     1     1     1     1     1 
28057  1061 12063  1063  1107 13099 13253 47051 13215  1083  1089  1075 28095 
    1     1     1     1     1     1     1     1     1     1     1     1     1 
28059  1099  1105 28087 13045 13053 28069  1123  1125  1129  1133  2020  2170 
    1     1     1     1     1     1     1     1     1     1     1     2     2 
 2122  4001  4017 35031 35045 49037  4003  4019  4023  4005  4007  4015  4013 
    2     3     3     3     3     3     3     3     3     3     3     3     3 
 4021  4012  4027  6065  6071 32003 49053  4025  5003  5139 22067  5007  5143 
    3     3     3     3     3     3     3     3     1     1     1     4     4 
29009 40001 40041  5017 28011  5027 22119  5031  5093  5033  5035  5123 28033 
    4     4     4     1     1     1     1     1     1     4     1     1     1 
28143 47157 47167  5037  5041  5079  5045  5023  5119  5051  5125  5053  5069 
    1     1     1     1     1     1     1     1     1     1     1     1     1 
 5055  5121  5061  5133  5063  5067  5073  5091  5075  5077  5095 22017 48037 
    1     1     1     1     1     1     1     1     1     1     1     1     1 
47097  5105  5107 28027  5111  5113 40089 22111  5145  5149  6001  6013  6075 
    1     1     1     1     1     1     1     1     1     1     3     3     3 
 6077  6081  6085  6009  6041  6067  6095  6017  6019  6039  6051  6053  6107 
    3     3     3     3     3     3     3     3     3     3     3     3     3 
 6025  6073  6027  6029 32023  6037  6083  6111  6031  6033  6055  6097  6113 
    3     3     3     3     3     3     3     3     3     3     3     3     3 
 6059  6045  6047  6087  6057 32031  6061  6069  6079  6099  6101  6109  6115 
    3     3     3     3     3     3     3     3     3     3     3     3     3 
 8001  8005  8014  8031  8059  8087  8123  8035  8039  8007 35039  8013  8069 
    3     3     3     3     3     3     3     3     3     3     3     3     3 
 8015  8043  8051  8029  8085  8041  8037  8045  8117  8101  8053  8065  8067 
    3     3     3     3     3     3     3     3     3     3     3     3     3 
56021  8071  8089 35007  8073  8075  8083  8113  8091  8095  8097  8105  8003 
    3     3     3     3     3     3     3     3     3     3     3     5     5 
 8107  8081  8119  8121  8125  9001  9005  9009 36027 36059 36079 36103 36119 
    3     3     3     3     3     1     1     1     1     1     1     1     1 
 9003  9007  9011  9013 25013  9015 44003 44009 25027 44007 10001 10003 10005 
    1     1     1     1     1     1     1     1     1     1     1     1     1 
24011 24029 24035 34009 34011 34033 24015 34015 42029 42045 24019 24045 24047 
    1     1     1     1     1     1     1     1     1     1     1     1     1 
11001 24031 24033 51013 51510 12001 12023 12083 12107 12005 12007 12019 12009 
    1     1     1     1     1     1     1     1     1     1     1     1     1 
12061 12095 12097 12127 12011 12021 12051 12086 12099 12013 12015 12071 12115 
    1     1     1     1     1     1     1     1     1     1     1     1     1 
12017 12119 12031 12109 12027 12055 12081 12029 12123 12089 12035 12039 12073 
    1     1     1     1     1     1     1     1     1     1     1     1     1 
13131 12041 12043 12047 12079 13185 12

In [43]:
##Betweenness

#This line calculates the betweenness of each county for our week 10 period
btwn.wk10 <- betweenness(week_13_14.g)

#This line assigns the corresponding name with its corresponding betweenness value
names(btwn.wk10) <- V(week_13_14.g)#county_states2$county[county_states2$fips %in% rownames(as.matrix(V(week_6_7.g)))]

#This line now rearrages the data in decreasing order of betweennees value
ind <- order(-btwn.wk10)

#This line allow us to see the top ten highest counties with betweenness values 
btwn.wk10[ind][1:10]

#The betweenness value is a value that allows us to see how often a county can be used as a link or gateway 
#between another county within the network

370      459      405      368      407      392      409      393 
49115.23 48403.21 48365.21 44757.04 41896.21 41613.96 38971.46 37381.67 
    1608     1612 
34233.43 31131.57

In [45]:
## Centrality 

#This line calculates the centrality of each faculty member within our initial data set
cent.eig <- evcent(week_13_14.g)

#This line assigns the corresponding name with its corresponding centrality value
names(cent.eig$vector) <- V(week_13_14.g) #county_states2$county[county_states2$fips %in% rownames(as.matrix(V(week_6_7.g)))]

#This line now rearrages the data in decreasing order of centrality value
ind <- order(-cent.eig$vector)

#This line allow us to see the top ten highest faculty members with centrality values 
cent.eig$vector[ind][1:10]

924       275       289       925       272       927       271       937 
1.0000000 0.9544352 0.9144751 0.8867406 0.8307476 0.8218892 0.8108864 0.7737880 
      932       938 
0.7573907 0.7263685

# ISSUES

Indexing the graphs objects to produce the proper county fips codes